In [9]:
import logging
import os
import sys
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from tqdm import tqdm

sys.path.append(os.path.abspath("../.."))
from spectrum.utils import set_random_state
from spectrum.models import SaVAE_SR
from spectrum.dataset import YahooDataset

logging.basicConfig(level=logging.INFO)

warnings.filterwarnings("ignore")

sns.set_theme(style="whitegrid")
plt.rcParams.update(
    {
        "axes.edgecolor": "0.3",
        "axes.linewidth": 0.8,
        "font.size": 12,
        "axes.titlesize": 14,
        "axes.labelsize": 12,
        "axes.titleweight": "bold",
        "legend.fontsize": 10,
        "figure.dpi": 120,
        "legend.frameon": False,
    }
)
set_random_state()

In [10]:
selected_ids = [4, 17, 33]

results_dir = "../../results/models/savae_sr"
os.makedirs(results_dir, exist_ok=True)


def find_best_threshold(scores, true_labels, thresholds=None):
    if thresholds is None:
        # use percentiles as candidate thresholds
        thresholds = [np.percentile(scores, p) for p in range(50, 100, 1)]
        # add some extra threshold points
        thresholds.extend([np.percentile(scores, p) for p in [99.5, 99.9]])

    best_f1 = 0
    best_threshold = thresholds[0]
    best_metrics = {}

    for threshold in thresholds:
        pred_labels = (scores > threshold).astype(int)

        # calculate confusion matrix
        TP = ((true_labels == 1) & (pred_labels == 1)).sum()
        FP = ((true_labels == 0) & (pred_labels == 1)).sum()
        TN = ((true_labels == 0) & (pred_labels == 0)).sum()
        FN = ((true_labels == 1) & (pred_labels == 0)).sum()

        # calculate metrics
        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        fnr = FN / (FN + TP) if (FN + TP) > 0 else 0
        fpr = FP / (FP + TN) if (FP + TN) > 0 else 0
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            best_metrics = {
                'threshold': threshold,
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'fnr': fnr,
                'fpr': fpr,
                'tp': TP,
                'fp': FP,
                'tn': TN,
                'fn': FN
            }

    return best_threshold, best_metrics


def process_single_id(data_id):
    print(f"processing: {data_id}")

    train_data = YahooDataset(
        _id=data_id,
    )
    test_data = YahooDataset(_id=data_id, path="../../datasets/Yahoo/test/")
    complete_data = pl.read_csv(f"../../datasets/Yahoo/data/A1Benchmark/{data_id}.csv")

    print("  training model...")
    model = SaVAE_SR()
    start_time = time.time()
    model.fit(train_data)
    training_time = time.time() - start_time

    print("  anomaly detection...")
    start_time = time.time()
    scores = model.predict(test_data)
    scoring_time = time.time() - start_time

    train_len = len(pl.read_csv(f"../../datasets/Yahoo/train/A1/{data_id}.csv")['value'])
    test_len = len(complete_data) - train_len
    test_true_labels = complete_data["label"][train_len:train_len + test_len].to_numpy()
    scores_array = scores

    print("  finding best threshold...")
    best_threshold, best_metrics = find_best_threshold(scores_array, test_true_labels)

    complete_values = complete_data["value"].to_numpy()
    complete_labels = complete_data["label"].to_numpy()
    complete_timestamps = complete_data["timestamp"] if "timestamp" in complete_data.columns else range(
        len(complete_values))

    complete_predictions = np.zeros(len(complete_values))
    complete_predictions[train_len:train_len + test_len] = (scores_array > best_threshold).astype(int)

    result_df = pd.DataFrame({
        'timestamp': complete_timestamps,
        'value': complete_values,
        'label': complete_labels,
        'predicted': complete_predictions,
        'anomaly_score': np.concatenate(
            [np.zeros(train_len), scores_array, np.zeros(len(complete_values) - train_len - test_len)])
    })

    output_file = os.path.join(results_dir, f"{data_id}.csv")
    result_df.to_csv(output_file, index=False)
    print(f"  results saved to: {output_file}")

    return {
        'id': data_id,
        'training_time': training_time,
        'testing_time': scoring_time,
        'total_time': training_time + scoring_time,
        'train_samples': len(train_data),
        'test_samples': test_len,
        'best_threshold': best_threshold,
        **best_metrics
    }


all_results = []
print(f"processing {len(selected_ids)} datasets...")

for data_id in tqdm(selected_ids, desc="processing"):
    try:
        result = process_single_id(data_id)
        all_results.append(result)

        print(f"  ID {data_id} completed:")
        print(f"    best_threshold: {result['best_threshold']:.4f}")
        print(f"    f1: {result['f1']:.4f}")
        print(f"    precision: {result['precision']:.4f}")
        print(f"    recall: {result['recall']:.4f}")
        print(f"    accuracy: {result['accuracy']:.4f}")

    except Exception as e:
        print(f"  processing {data_id} failed: {str(e)}")
        import traceback

        traceback.print_exc()
        continue

if all_results:
    summary_df = pd.DataFrame(all_results)
    summary_file = os.path.join(results_dir, "savae_sr.csv")
    summary_df.to_csv(summary_file, index=False)
    print(f"summary results saved to: {summary_file}")

    print("\n" + "=" * 80)
    print("LSTM anomaly detection results")
    print("=" * 80)
    print(f"processed {len(all_results)} datasets")
    print(f"average F1: {summary_df['f1'].mean():.4f} ± {summary_df['f1'].std():.4f}")
    print(f"average precision: {summary_df['precision'].mean():.4f} ± {summary_df['precision'].std():.4f}")
    print(f"average recall: {summary_df['recall'].mean():.4f} ± {summary_df['recall'].std():.4f}")
    print(f"average accuracy: {summary_df['accuracy'].mean():.4f} ± {summary_df['accuracy'].std():.4f}")
    print(f"average training time: {summary_df['training_time'].mean():.2f}s")
    print(f"average scoring time: {summary_df['testing_time'].mean():.2f}s")
    print("=" * 80)

    print("details:")
    display_cols = ['id', 'f1', 'precision', 'recall', 'accuracy', 'best_threshold', 'tp', 'fp', 'tn', 'fn']
    print(summary_df[display_cols].round(4))
else:
    print("no results")

processing 3 datasets...


processing:   0%|          | 0/3 [00:00<?, ?it/s]

processing: 4
  training model...



Epoch 1/60: 100%|██████████| 2/2 [00:01<00:00,  1.76it/s, loss=13.5]

Epoch 2/60: 100%|██████████| 2/2 [00:00<00:00,  2.06it/s, loss=13.5]

Epoch 3/60: 100%|██████████| 2/2 [00:00<00:00, 35.28it/s, loss=13.5]

Epoch 4/60: 100%|██████████| 2/2 [00:00<00:00, 33.22it/s, loss=13.4]

Epoch 5/60: 100%|██████████| 2/2 [00:00<00:00, 30.35it/s, loss=13.5]

Epoch 6/60: 100%|██████████| 2/2 [00:00<00:00, 36.58it/s, loss=13.6]

Epoch 7/60: 100%|██████████| 2/2 [00:00<00:00, 36.34it/s, loss=13.6]

Epoch 8/60: 100%|██████████| 2/2 [00:00<00:00, 36.59it/s, loss=13.6]

Epoch 9/60: 100%|██████████| 2/2 [00:00<00:00, 35.66it/s, loss=13.5]

Epoch 10/60: 100%|██████████| 2/2 [00:00<00:00, 34.79it/s, loss=13.5]

Epoch 11/60: 100%|██████████| 2/2 [00:00<00:00, 36.66it/s, loss=13.5]

Epoch 12/60: 100%|██████████| 2/2 [00:00<00:00, 34.78it/s, loss=13.6]

Epoch 13/60: 100%|██████████| 2/2 [00:00<00:00, 33.17it/s, loss=13.5]

Epoch 14/60: 100%|██████████| 2/2 [00:00<00:00, 35.81it/s, loss=13.5]

Epoch 15/60: 1

  anomaly detection...


...

  finding best threshold...
  results saved to: ../../results/models/savae_sr/17.csv
  ID 17 completed:
    best_threshold: 0.7859
    f1: 0.5385
    precision: 0.6140
    recall: 0.4795
    accuracy: 0.8315
processing: 33
  training model...



Epoch 1/60: 100%|██████████| 2/2 [00:00<00:00, 21.14it/s, loss=13.5]

Epoch 2/60: 100%|██████████| 2/2 [00:00<00:00, 21.95it/s, loss=13.4]

Epoch 3/60: 100%|██████████| 2/2 [00:00<00:00, 28.18it/s, loss=13.5]

Epoch 4/60: 100%|██████████| 2/2 [00:00<00:00, 33.01it/s, loss=13.5]

Epoch 5/60: 100%|██████████| 2/2 [00:00<00:00, 29.83it/s, loss=13.4]

Epoch 6/60: 100%|██████████| 2/2 [00:00<00:00, 32.44it/s, loss=13.4]

Epoch 7/60: 100%|██████████| 2/2 [00:00<00:00, 31.84it/s, loss=13.5]

Epoch 8/60: 100%|██████████| 2/2 [00:00<00:00, 32.96it/s, loss=13.5]

Epoch 9/60: 100%|██████████| 2/2 [00:00<00:00, 27.11it/s, loss=13.5]

Epoch 10/60: 100%|██████████| 2/2 [00:00<00:00, 29.37it/s, loss=13.5]

Epoch 11/60: 100%|██████████| 2/2 [00:00<00:00,  2.26it/s, loss=13.4]

Epoch 12/60: 100%|██████████| 2/2 [00:01<00:00,  1.97it/s, loss=13.5]

Epoch 13/60: 100%|██████████| 2/2 [00:00<00:00,  2.64it/s, loss=13.4]

Epoch 14/60: 100%|██████████| 2/2 [00:00<00:00, 31.21it/s, loss=13.5]

Epoch 15/60: 1

  anomaly detection...


processing: 100%|██████████| 3/3 [00:28<00:00,  9.39s/it]

  finding best threshold...
  results saved to: ../../results/models/savae_sr/33.csv
  ID 33 completed:
    best_threshold: 0.7544
    f1: 1.0000
    precision: 1.0000
    recall: 1.0000
    accuracy: 1.0000
summary results saved to: ../../results/models/savae_sr/savae_sr.csv

LSTM anomaly detection results
processed 3 datasets
average F1: 0.7179 ± 0.2473
average precision: 0.8713 ± 0.2228
average recall: 0.6413 ± 0.3111
average accuracy: 0.9415 ± 0.0953
average training time: 8.98s
average scoring time: 0.39s
details:
   id      f1  precision  recall  accuracy  best_threshold  tp  fp   tn  fn
0   4  0.6154      1.000  0.4444    0.9930          0.6088   4   0  703   5
1  17  0.5385      0.614  0.4795    0.8315          0.7859  70  44  522  76
2  33  1.0000      1.000  1.0000    1.0000          0.7544   1   0  719   0
